<a href="https://colab.research.google.com/github/injeela-ansari/Disease-Prediction-App/blob/main/merged_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas scikit-learn streamlit streamlit-option-menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.0 MB/s eta 0:00:00


In [5]:
# app.py

import pickle
import joblib
import streamlit as st
from streamlit_option_menu import option_menu
import numpy as np
import pandas as pd

# --- SETTINGS ---
page_title = "Health Prediction Suite"
page_icon = "❤️‍🩹"
layout = "centered"

st.set_page_config(page_title=page_title, page_icon=page_icon, layout=layout)
st.title(page_title + " " + page_icon)

# --- LOADING THE SAVED MODELS AND SCALERS ---
try:
    # Disease Models
    diabetes_model = pickle.load(open('diabetes_model.sav', 'rb'))
    diabetes_scaler = pickle.load(open('diabetes_scaler.sav', 'rb'))
    heart_disease_model = pickle.load(open('heart_disease_model.sav', 'rb'))
    heart_disease_scaler = pickle.load(open('heart_disease_scaler.sav', 'rb'))
    parkinsons_model = pickle.load(open('parkinsons_model.sav', 'rb'))
    parkinsons_scaler = pickle.load(open('parkinsons_scaler.sav', 'rb'))

    # Insurance Model
    insurance_model_pipeline = joblib.load('insurance_model_pipeline.joblib')

except FileNotFoundError as e:
    st.error(f"Error loading model files: {e}. Please ensure all model and scaler files are in the correct directory.")
    st.stop()
except Exception as e:
    st.error(f"An unexpected error occurred: {e}")
    st.stop()

# --- SIDEBAR FOR NAVIGATION ---
with st.sidebar:
    selected = option_menu('Health Prediction Suite',
                           ['Diabetes Prediction', 'Heart Disease Prediction', 'Parkinsons Prediction', 'Insurance Cost Prediction'],
                           icons=['bi-activity', 'bi-heart-pulse', 'bi-person', 'bi-shield-check'],
                           menu_icon='hospital-fill',
                           default_index=0)

# --- DIABETES PREDICTION PAGE ---
if selected == 'Diabetes Prediction':
    st.header('Diabetes Prediction')
    st.markdown("Enter the patient's details below:")

    col1, col2, col3 = st.columns(3)
    with col1: Pregnancies = st.text_input('Number of Pregnancies')
    with col2: Glucose = st.text_input('Glucose Level')
    with col3: BloodPressure = st.text_input('Blood Pressure value')
    with col1: SkinThickness = st.text_input('Skin Thickness value')
    with col2: Insulin = st.text_input('Insulin Level')
    with col3: BMI = st.text_input('BMI value')
    with col1: DiabetesPedigreeFunction = st.text_input('Diabetes Pedigree Function')
    with col2: Age = st.text_input('Age')

    if st.button('Predict Diabetes'):
        try:
            user_input = np.array([[Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age]], dtype=float)
            user_input_scaled = diabetes_scaler.transform(user_input)
            prediction = diabetes_model.predict(user_input_scaled)

            st.subheader("Prediction Result:")
            if prediction[0] == 1:
                st.error('The person is predicted to be **Diabetic**.')
            else:
                st.success('The person is predicted to be **Not Diabetic**.')
        except (ValueError, TypeError):
            st.warning("Please enter valid numerical data in all fields.")

# --- HEART DISEASE PREDICTION PAGE ---
if selected == 'Heart Disease Prediction':
    st.header('Heart Disease Prediction')
    st.markdown("Enter the patient's details below:")

    col1, col2, col3 = st.columns(3)
    with col1: age = st.text_input('Age')
    with col2: sex = st.selectbox('Sex', ['Male', 'Female'])
    with col3: cp = st.selectbox('Chest Pain Type', ['Typical Angina', 'Atypical Angina', 'Non-anginal Pain', 'Asymptomatic'])
    with col1: trestbps = st.text_input('Resting Blood Pressure')
    with col2: chol = st.text_input('Serum Cholestoral (mg/dl)')
    with col3: fbs = st.selectbox('Fasting Blood Sugar > 120 mg/dl', ['True', 'False'])
    with col1: restecg = st.selectbox('Resting ECG Results', ['Normal', 'ST-T wave abnormality', 'Left ventricular hypertrophy'])
    with col2: thalach = st.text_input('Max Heart Rate Achieved')
    with col3: exang = st.selectbox('Exercise Induced Angina', ['Yes', 'No'])
    with col1: oldpeak = st.text_input('ST depression induced by exercise')
    with col2: slope = st.selectbox('Slope of peak exercise ST segment', ['Upsloping', 'Flat', 'Downsloping'])
    with col3: ca = st.text_input('Number of major vessels (0-3)')
    with col1: thal = st.selectbox('Thalassemia', ['Normal', 'Fixed defect', 'Reversable defect'])

    if st.button('Predict Heart Disease'):
        try:
            sex_val = 1 if sex == 'Male' else 0
            cp_val = ['Typical Angina', 'Atypical Angina', 'Non-anginal Pain', 'Asymptomatic'].index(cp)
            fbs_val = 1 if fbs == 'True' else 0
            restecg_val = ['Normal', 'ST-T wave abnormality', 'Left ventricular hypertrophy'].index(restecg)
            exang_val = 1 if exang == 'Yes' else 0
            slope_val = ['Upsloping', 'Flat', 'Downsloping'].index(slope)
            thal_val = ['Normal', 'Fixed defect', 'Reversable defect'].index(thal) + 1

            user_input = np.array([[age, sex_val, cp_val, trestbps, chol, fbs_val, restecg_val, thalach, exang_val, oldpeak, slope_val, ca, thal_val]], dtype=float)
            user_input_scaled = heart_disease_scaler.transform(user_input)
            prediction = heart_disease_model.predict(user_input_scaled)

            st.subheader("Prediction Result:")
            if prediction[0] == 1:
                st.error('The person is predicted to have **Heart Disease**.')
            else:
                st.success('The person is predicted to be **Healthy**.')
        except (ValueError, TypeError):
            st.warning("Please enter valid numerical data in all fields.")

# --- PARKINSON'S PREDICTION PAGE ---
if selected == "Parkinsons Prediction":
    st.header("Parkinson's Disease Prediction")
    st.markdown("Enter the patient's vocal measurements below:")

    # Using 4 columns for a better layout
    col1, col2, col3, col4 = st.columns(4)
    with col1: fo = st.text_input('MDVP:Fo(Hz)')
    with col2: fhi = st.text_input('MDVP:Fhi(Hz)')
    with col3: flo = st.text_input('MDVP:Flo(Hz)')
    with col4: Jitter_percent = st.text_input('MDVP:Jitter(%)')
    with col1: Jitter_Abs = st.text_input('MDVP:Jitter(Abs)')
    with col2: RAP = st.text_input('MDVP:RAP')
    with col3: PPQ = st.text_input('MDVP:PPQ')
    with col4: DDP = st.text_input('Jitter:DDP')
    with col1: Shimmer = st.text_input('MDVP:Shimmer')
    with col2: Shimmer_dB = st.text_input('MDVP:Shimmer(dB)')
    with col3: APQ3 = st.text_input('Shimmer:APQ3')
    with col4: APQ5 = st.text_input('Shimmer:APQ5')
    with col1: APQ = st.text_input('MDVP:APQ')
    with col2: DDA = st.text_input('Shimmer:DDA')
    with col3: NHR = st.text_input('NHR')
    with col4: HNR = st.text_input('HNR')
    with col1: RPDE = st.text_input('RPDE')
    with col2: DFA = st.text_input('DFA')
    with col3: spread1 = st.text_input('spread1')
    with col4: spread2 = st.text_input('spread2')
    with col1: D2 = st.text_input('D2')
    with col2: PPE = st.text_input('PPE')

    if st.button("Predict Parkinson's"):
        try:
            user_input = np.array([[fo, fhi, flo, Jitter_percent, Jitter_Abs, RAP, PPQ, DDP,Shimmer, Shimmer_dB, APQ3, APQ5, APQ, DDA, NHR, HNR, RPDE, DFA, spread1, spread2, D2, PPE]], dtype=float)
            user_input_scaled = parkinsons_scaler.transform(user_input)
            prediction = parkinsons_model.predict(user_input_scaled)

            st.subheader("Prediction Result:")
            if prediction[0] == 1:
                st.error("The person is predicted to have **Parkinson's Disease**.")
            else:
                st.success("The person is predicted to be **Healthy**.")
        except (ValueError, TypeError):
            st.warning("Please enter valid numerical data in all fields.")

# --- INSURANCE COST PREDICTION PAGE ---
if selected == 'Insurance Cost Prediction':
    st.header("Medical Insurance Cost Predictor 💵")
    st.markdown("Enter your details below to estimate your annual medical insurance cost.")

    # Using columns for a cleaner layout
    col1, col2 = st.columns(2)
    with col1:
        age = st.number_input('Age', min_value=18, max_value=100, value=30, step=1)
        bmi = st.number_input('Body Mass Index (BMI)', min_value=15.0, max_value=60.0, value=25.0, step=0.1)
        sex = st.radio('Sex', ['male', 'female'])
    with col2:
        children = st.selectbox('Number of Children', [0, 1, 2, 3, 4, 5])
        smoker = st.radio('Smoker', ['yes', 'no'])
        region = st.selectbox('Region', ['southwest', 'southeast', 'northwest', 'northeast'])

    if st.button('**Predict Insurance Cost**'):
        try:
            input_data = pd.DataFrame({
                'age': [age],
                'sex': [sex],
                'bmi': [bmi],
                'children': [children],
                'smoker': [smoker],
                'region': [region]
            })

            prediction = insurance_model_pipeline.predict(input_data)

            st.subheader("✨ Predicted Insurance Cost")
            st.markdown(f"## **${prediction[0]:,.2f}**")
            st.info("This is an estimated cost. Actual costs may vary.")
        except Exception as e:
            st.error(f"An error occurred during prediction: {e}")

2025-09-05 18:15:52.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.530 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 18:15:52.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar